In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import codecs,csv

In [11]:
def listdir(path):  #传入存储的list
    list_path = []
    list_name = []
    for file in os.listdir(path):  
        file_path = os.path.join(path, file)
        list_name.append(os.path.splitext(file)[0])
        if os.path.isdir(file_path):  
            listdir(file_path, list_path)  
        else:  
            list_path.append(file_path)
    return list_path,list_name

filepathes,filenames = listdir(".\data\豆瓣标签")
file_data = pd.DataFrame({'path':filepathes,'name':filenames})
book_num = len(filepathes)
# filenames = pd.DataFrame(filenames,columns=[])
# filenames
file_data

,path,name
0,.\data\豆瓣标签\100层的房子.txt,100层的房子
1,.\data\豆瓣标签\1900年以来的室内设计（增订版）.txt,1900年以来的室内设计（增订版）
2,.\data\豆瓣标签\1984.txt,1984
3,.\data\豆瓣标签\1988：我想和这个世界谈谈.txt,1988：我想和这个世界谈谈
4,.\data\豆瓣标签\1999年6月29日.txt,1999年6月29日
...,...,...
1411,.\data\豆瓣标签\黄昏里的男孩.txt,黄昏里的男孩
1412,.\data\豆瓣标签\黄金时代.txt,黄金时代
1413,.\data\豆瓣标签\黎明.txt,黎明
1414,.\data\豆瓣标签\黑兔和白兔.txt,黑兔和白兔


In [5]:
tags = []
data_columns = []
for i in range(len(file_data)):
    tag=pd.read_table(file_data.path[i],sep='\t',header=None,encoding='GBK',names='T')
    tags.append(tag['T'].tolist())
for  i in tags :
    for j in i:
        if j not in data_columns:
            data_columns.append(j)
bookdata = pd.DataFrame(list(np.zeros([book_num,len(data_columns)])),columns=data_columns)  
for  i in tags :
    for j in i:
        bookdata.loc[tags.index(i),j] = bookdata.loc[tags.index(i),j]+1
bookdata.to_csv('./bookdata.csv',encoding='gbk')

In [6]:
datahang = bookdata.sum(axis=0)
data_sum = pd.DataFrame( {'name': datahang.index, 'total':datahang.values})
data_sum

,name,total
0,绘本,486.0
1,日本,148.0
2,童书,375.0
3,岩井俊雄,3.0
4,儿童绘本,83.0
...,...,...
2757,爱心树绘本馆,1.0
2758,（美）乌利·舒利瓦茨,1.0
2759,黑兔与白兔,1.0
2760,兔子,1.0


In [9]:
temp = data_sum.loc[data_sum["total"] > 20]
recommendation_columns = temp['name'].tolist()
item_recommendation = [[item,[]] for item in recommendation_columns]
for  i in tags :
    for j in i:
        if j in recommendation_columns:
            item_recommendation[recommendation_columns.index(j)][1].append(file_data['name'][tags.index(i)])

def data_write_csv(file_name, datas):#file_name为写入CSV文件的路径，datas为要写入数据列表
    file_csv = codecs.open(file_name,'w+','utf-8')#追加
    writer = csv.writer(file_csv, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
    for data in datas:
        writer.writerow(data)
    print("保存文件成功，处理结束")
    file_csv.close()
data_write_csv('rec.csv',item_recommendation)      


保存文件成功，处理结束


In [15]:
def text_save(filename, data):#filename为写入CSV文件的路径，data为要写入数据列表.
    file = open(filename,'a')
    for i in range(len(data)):
        str1 = str(data[i][0]) + ' '
        str2 = ''
        for j in range(len(data[i])-1):
            str2 = str2 + str(data[i][j])+ "," 
        str2 = str2 + str(data[i][len(data[i])-1]) + '\n'
        file.write(s)
    file.close()
    print("保存文件成功")
data_write_csv('rec.txt',item_recommendation)

保存文件成功，处理结束


In [12]:
#!/usr/bin/env python
#-*-coding:utf-8-*-
import random  
import math
#统计各类数量  
def addValueToMat(theMat,key,value,incr):  
    if key not in theMat: #如果key没出先在theMat中  
        theMat[key]=dict();  
        theMat[key][value]=incr;  
    else:  
        if value not in theMat[key]:  
            theMat[key][value]=incr;  
        else:  
            theMat[key][value]+=incr;#若有值，则递增  
  
user_tags = dict();  
tag_items = dict();  
user_items = dict();  
user_items_test = dict();#测试集数据字典  
item_tags = dict()        #用于多样性测试
  
#初始化，进行各种统计  
def InitStat():  
    global user_tags,tag_items,user_items,user_items_test,item_tags
    data_file = open('delicious.dat',encoding='gb2312')  
    line = data_file.readline();   
    while line:  
        if random.random()>0.1:#将90%的数据作为训练集，剩下10%的数据作为测试集  
            terms = line.split("\t");#训练集的数据结构是[user, item, tag]形式  
            user=terms[0];  
            item=terms[1];  
            tag=terms[2];  
            addValueToMat(user_tags,user,tag,1)  
            addValueToMat(tag_items,tag,item,1)  
            addValueToMat(user_items,user,item,1)
            addValueToMat(item_tags,item,tag,1)  
            line = data_file.readline();  
        else:  
            addValueToMat(user_items_test,user,item,1)  
    data_file.close();     
    
#推荐算法  
def Recommend(usr):  
    recommend_list = dict();  
    tagged_item = user_items[usr];#得到该用户所有推荐过的物品  
    for tag_,wut in user_tags[usr].items():#用户打过的标签及次数  
        for item_,wit in tag_items[tag_].items():#物品被打过的标签及被打过的次数  
            if item_ not in tagged_item:#已经推荐过的不再推荐  
                if item_ not in recommend_list:  
                    recommend_list[item_]=wut*wit;#根据公式  
                else:  
                    recommend_list[item_]+=wut*wit;  
    return sorted(recommend_list.iteritems(), key=lambda a:a[1],reverse=True)

#统计标签流行度
def TagPopularity():
    tagfreq = {}
    for user in user_tags.keys():
        for tag in user_tags[user].keys():
            if tag not in tagfreq:
                tagfreq[tag] = 1
            else:
                tagfreq[tag] +=1
    return sorted(tagfreq.iteritems(), key=lambda a:a[1],reverse=True)

#计算余弦相似度
def CosineSim(item_tags,i,j):
    ret = 0
    for b,wib in item_tags[i].items():     #求物品i,j的标签交集数目
        if b in item_tags[j]:
            ret += wib * item_tags[j][b]
    ni = 0
    nj = 0
    for b, w in item_tags[i].items():      #统计 i 的标签数目
        ni += w * w
    for b, w in item_tags[j].items():      #统计 j 的标签数目
        nj += w * w
    if ret == 0:
        return 0
    return ret/math.sqrt(ni * nj)          #返回余弦值       

#计算推荐列表多样性
def Diversity(item_tags,recommend_items):
    ret = 0
    n = 0
    for i in dict(recommend_items).keys():
        for j in dict(recommend_items).keys():
            if i == j:
                continue
            ret += CosineSim(item_tags,i,j)
            n += 1
    return ret/(n * 1.0)

InitStat()
recommend_list = Recommend("48411")
# print recommend_list
for recommend in recommend_list[:10]:  #兴趣度最高的十个itemid
    print (recommend)

#标签流行度统计    
tagFreq = TagPopularity()
for tag in tagFreq[:20]:
    print (tag)

#推荐列表多样性,计算时间较长
diversityNum = Diversity(item_tags, recommend_list)
print (diversityNum)